In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torchvision.transforms as transforms
from torchvision.models.video import R3D_18_Weights, MC3_18_Weights
from torchvision.models.video import R2Plus1D_18_Weights, S3D_Weights
from torchvision.models.video import MViT_V2_S_Weights, MViT_V1_B_Weights
from torchvision.models.video import mvit_v2_s, MViT_V2_S_Weights, mvit_v1_b, MViT_V1_B_Weights
import torch

from src.dataset import MultiViewDataset
from src.mvnetwork import MVNetwork
from src.loss import calculate_outputs, calculate_loss, get_criterion
from src.model import LitMVNNetwork
import lightning as L

path = "data/mvfouls"
start_frame = 0
end_frame = 125 # TODO - make lower
fps = 25
num_views = 5
pre_model = "s3d"
max_num_worker = 8
batch_size = 2
data_aug = False
pooling_type = 'max'
weight_decay = 0.001
step_size = 3
gamma = 0.1
LR = 0.01
weighted_loss = False

if data_aug == 'Yes':
        transformAug = transforms.Compose([
                                          transforms.RandomAffine(degrees=(0, 0), translate=(0.1, 0.1), scale=(0.9, 1)),
                                          transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
                                          transforms.RandomRotation(degrees=5),
                                          transforms.ColorJitter(brightness=0.5, saturation=0.5, contrast=0.5),
                                          transforms.RandomHorizontalFlip()
                                          ])
else:
    transformAug = None

if pre_model == "r3d_18":
    transforms_model = R3D_18_Weights.KINETICS400_V1.transforms()        
elif pre_model == "s3d":
    transforms_model = S3D_Weights.KINETICS400_V1.transforms()       
elif pre_model == "mc3_18":
    transforms_model = MC3_18_Weights.KINETICS400_V1.transforms()       
elif pre_model == "r2plus1d_18":
    transforms_model = R2Plus1D_18_Weights.KINETICS400_V1.transforms()
elif pre_model == "mvit_v2_s":
    transforms_model = MViT_V2_S_Weights.KINETICS400_V1.transforms()

dataset_Train = MultiViewDataset(path=path, start=start_frame, end=64, fps=fps, split='train',
            num_views = 1, transform=transformAug, transform_model=transforms_model)
criterion = get_criterion(weighted_loss, dataset_Train)

5277


In [9]:
print(len(dataset_Train))

5277


In [4]:
train_loader = torch.utils.data.DataLoader(dataset_Train,
            batch_size=batch_size, shuffle=True,
            num_workers=max_num_worker)

In [5]:
counter = 0
for targets_offence_severity, targets_action, mvclips, action in train_loader:
    print(mvclips.shape)
    counter += 1
    if counter == 5:
        break

ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 

RuntimeError: DataLoader worker (pid(s) 80434) exited unexpectedly

In [9]:
# model = MVNetwork(net_name=pre_model, agr_type=pooling_type).cuda()

# optimizer = torch.optim.AdamW(model.parameters(), lr=LR, 
#                                     betas=(0.9, 0.999), eps=1e-07, 
#                                     weight_decay=weight_decay, amsgrad=False)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# actions = {}

# for targets_offence_severity, targets_action, mvclips, action in train_loader:
#     targets_offence_severity = targets_offence_severity.cuda()
#     targets_action = targets_action.cuda()
#     mvclips = mvclips.cuda().float()
#     outputs_offence_severity, outputs_action, _ = model(mvclips)
#     outputs_offence_severity, outputs_action, actions = calculate_outputs(outputs_offence_severity, outputs_action, action, actions)
#     loss = calculate_loss(criterion, outputs_offence_severity, outputs_action, targets_offence_severity, targets_action)
#     print(loss)
#     break


model = LitMVNNetwork(pre_model=pre_model, pooling_type=pooling_type, criterion=criterion)

In [10]:
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=model, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type      | Params
------------------------------------
0 | model | MVNetwork | 9.0 M 
------------------------------------
9.0 M     Trainable params
0         Non-trainable params
9.0 M     Total params
35.875    Total estimated model params size (MB)
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Epoch 0:   0%|          | 0/100 [00:00<?, ?it/s] 

/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. P

x
x
x
Epoch 0:   1%|          | 1/100 [00:03<05:25,  0.30it/s, v_num=7]

RuntimeError: Caught RuntimeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 277, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 144, in collate
    return [collate(samples